In [0]:
# === Install Dependencies ===
%pip install lightgbm scikit-learn joblib pandas numpy

Python interpreter will be restarted.
Python interpreter will be restarted.


In [0]:
# === Import Libraries ===
import os, re
from pyspark.sql import SparkSession
from pyspark.sql.functions import (
    col, expr, when, lag, avg, stddev_samp,
    min as min_, max as max_, first, count, last, lit
)
from pyspark.sql.window import Window
from pyspark.sql.types import IntegerType, DoubleType
from pyspark.storagelevel import StorageLevel
import pandas as pd
import numpy as np
import joblib
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error

In [0]:
# === Establish SparkSession with Configuration ===
spark = SparkSession.builder.appName("M5_LightGBM_SinglePeriod") \
    .config("spark.sql.autoBroadcastJoinThreshold", "-1") \
    .config("spark.driver.maxResultSize", "8g") \
    .getOrCreate()

In [0]:
# === Load Raw Data ===
sales = spark.read.option("header", True).csv("/FileStore/sales_train_evaluation.csv")
calendar = spark.read.option("header", True).csv("/FileStore/calendar.csv")
prices = spark.read.option("header", True).csv("/FileStore/sell_prices.csv")

In [0]:
# === Calendar Features ===
calendar_sel = calendar.select(
    'd',
    col('wm_yr_wk').cast("int"),
    col('wday').cast("int"),
    col('month').cast("int"),
    col('year').cast("int"),
    col('snap_CA').cast("int"),
    col('snap_TX').cast("int"),
    col('snap_WI').cast("int"),
    when(col('event_name_1').isNotNull() | col('event_name_2').isNotNull(), 1).otherwise(0).alias('is_event'),
    when(col('wday').isin(1, 2), 1).otherwise(0).alias('is_weekend')
).withColumn('day_index', expr("cast(substring(d,3) as int)"))

In [0]:
# === Create a Full Grid for All Days (d1 to d1969) ===
items_stores = sales.select('item_id', 'store_id', 'dept_id', 'cat_id', 'state_id').distinct()
full_grid = items_stores.crossJoin(calendar_sel)

In [0]:
# === Melt Sales Data ===
sales_melted = sales.selectExpr(
    "item_id", "dept_id", "cat_id", "store_id", "state_id",
    "stack(" + str(len([c for c in sales.columns if c.startswith("d_")])) + ", " +
    ",".join([f"'{c}', {c}" for c in sales.columns if c.startswith("d_")]) +
    ") as (d, sales)"
).withColumn('day_index', expr("cast(substring(d,3) as int)"))

In [0]:
# === Join Sales with Full Grid ===
base_df = full_grid.join(
    sales_melted.select('item_id', 'store_id', 'day_index', 'sales'),
    on=['item_id', 'store_id', 'day_index'],
    how='left'
)

In [0]:
# === Price Features ===
prices_sel = prices.select(
    'item_id', 'store_id',
    col('wm_yr_wk').cast("int"),
    col('sell_price').cast("double").alias('price')
)
base_df = base_df.join(prices_sel, on=['item_id', 'store_id', 'wm_yr_wk'], how='left')

In [0]:
# === Remove Non-Selling Periods (Adjusted to preserve prediction range) ===
win0 = Window.partitionBy('item_id', 'store_id').orderBy('day_index')
base_df = base_df.withColumn(
    'first_day',
    min_(when(col('price').isNotNull(), col('day_index'))).over(win0)
).filter(col('day_index') >= col('first_day') - 28)
base_df = base_df.drop('first_day')

In [0]:
# === Geo Feature ===
base_df = base_df.withColumn('Geo', col('state_id'))

In [0]:
# === Define Windows ===
win_lag = Window.partitionBy('item_id', 'store_id').orderBy('day_index')
win_28rows = win_lag.rowsBetween(-27, 0)
win_all = Window.partitionBy('item_id', 'store_id')
win_cum = Window.partitionBy('item_id', 'store_id') \
                .orderBy('day_index') \
                .rowsBetween(Window.unboundedPreceding, 0)

In [0]:
# === Calculate Price Features ===
base_df = base_df.withColumn('lag1_price', lag('price', 1).over(win_lag))
base_df = base_df.withColumn('lag1_price',
    when(col('lag1_price').isNull(), col('price')).otherwise(col('lag1_price'))
).withColumn('price_diff_value', col('price') - col('lag1_price')) \
 .withColumn('price_diff_percentage',
    when(col('lag1_price') != 0,
         (col('price') - col('lag1_price')) / col('lag1_price') * 100
    ).otherwise(0)
).withColumn('cume_avg_price', avg('price').over(win_cum)) \
 .withColumn('cnt_window', count('price').over(win_28rows)) \
 .withColumn('d28_moving_avg_price',
    when(col('cnt_window') == 28, avg('price').over(win_28rows))
    .otherwise(col('cume_avg_price'))
 ).withColumn('max_price', max_('price').over(win_all)) \
 .withColumn('min_price', min_('price').over(win_all)) \
 .withColumn('mean_price', avg('price').over(win_all)) \
 .withColumn('stddev_price', stddev_samp('price').over(win_all))

In [0]:
# === Calculate Sales Features ===
base_df = base_df.withColumn('sales', col('sales').cast("double")) \
 .withColumn('lag1_sales', lag('sales', 1).over(win_lag))
avg_sales = base_df.groupBy('item_id', 'store_id').agg(avg('sales').alias('avg_sales'))
base_df = base_df.join(avg_sales, on=['item_id', 'store_id'], how='left') \
 .withColumn('lag1_sales',
    when(col('lag1_sales').isNull() & (col('day_index') <= 1941), col('avg_sales')) \
    .otherwise(col('lag1_sales'))
 )

In [0]:
# === Impute Sales for Future Days (d1942-d1969) ===
# Note: We impute sales for d1942-d1969 to compute features like lag1_sales and d28_moving_avg_sales.
# These imputed values are NOT the final predictions but are needed to generate features for the prediction period.
base_df = base_df.withColumn('sales', 
    when(col('day_index') > 1941, lit(None).cast("double")) \
    .otherwise(col('sales'))
).withColumn('sales', 
    when(col('day_index') > 1941, last('sales', ignorenulls=True).over(win_lag)) \
    .otherwise(col('sales'))
).withColumn('lag1_sales', 
    when(col('day_index') > 1941, last('lag1_sales', ignorenulls=True).over(win_lag)) \
    .otherwise(col('lag1_sales'))
 ) \
 .withColumn('cume_avg_sales', avg('sales').over(win_cum)) \
 .withColumn('d28_moving_avg_sales',
    when(col('cnt_window') == 28, avg('sales').over(win_28rows)) \
    .otherwise(col('cume_avg_sales'))
 ) \
 .withColumn('max_sales', max_('sales').over(win_all)) \
 .withColumn('min_sales', min_('sales').over(win_all)) \
 .withColumn('mean_sales', avg('sales').over(win_all)) \
 .withColumn('stddev_sales', stddev_samp('sales').over(win_all))

In [0]:
# === Handle Missing Values ===
base_df = base_df.fillna({'stddev_price': 0, 'stddev_sales': 0})

In [0]:
# === Build Single Training DataFrame (d1014-d1913) ===
train_start_idx = 1913 - 365  # Approximately one year before d1914
training_df = base_df.filter((col('day_index') >= train_start_idx) & (col('day_index') <= 1913)).select(
    expr("concat(item_id,'_',store_id,'_validation') as id"),
    'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'Geo',
    col('wm_yr_wk').cast("int"),
    col('wday').cast("int"),
    col('is_weekend').cast("int"),
    col('month').cast("int"),
    col('year').cast("int"),
    col('is_event').cast("int"),
    col('snap_CA').cast("int"),
    col('snap_TX').cast("int"),
    col('snap_WI').cast("int"),
    col('price').cast("double"),
    col('max_price').cast("double"),
    col('min_price').cast("double"),
    col('mean_price').cast("double"),
    col('stddev_price').cast("double"),
    col('d28_moving_avg_price').cast("double"),
    col('lag1_price').cast("double"),
    col('price_diff_value').cast("double"),
    col('price_diff_percentage').cast("double"),
    col('max_sales').cast("double"),
    col('min_sales').cast("double"),
    col('mean_sales').cast("double"),
    col('stddev_sales').cast("double"),
    col('d28_moving_avg_sales').cast("double"),
    col('lag1_sales').cast("double"),
    col('sales').cast("double"),
    'day_index'
).repartition('cat_id').persist(StorageLevel.MEMORY_AND_DISK)

In [0]:
# === Define Columns for Training ===
categories = ['FOODS', 'HOUSEHOLD', 'HOBBIES']
cat_cols = ['state_id', 'store_id', 'dept_id', 'Geo', 'wday', 'is_weekend', 'month', 'year', 'is_event', 'snap_CA', 'snap_TX', 'snap_WI']
num_cols = [
    'wm_yr_wk', 'price', 'max_price', 'min_price', 'mean_price', 'stddev_price',
    'd28_moving_avg_price', 'lag1_price', 'price_diff_value', 'price_diff_percentage',
    'max_sales', 'min_sales', 'mean_sales', 'stddev_sales', 'd28_moving_avg_sales', 'lag1_sales'
]
feat_cols = num_cols + cat_cols

In [0]:
# === Train Model and Predict for d1914–d1969 (Batch Processing) ===
def process_batch(train_df, pred_df, batch_size=5000):
    # Estimate splits without count() to reduce driver load
    train_weights = [float(1.0 / batch_size)] * batch_size
    pred_weights = [float(1.0 / batch_size)] * batch_size
    
    # Convert to Pandas in batches
    pdf_train_list = [batch.toPandas() for batch in train_df.randomSplit(train_weights, seed=42)]
    pdf_pred_list = [batch.toPandas() for batch in pred_df.randomSplit(pred_weights, seed=42)]
    
    # Train model on concatenated training batches
    pdf_train = pd.concat(pdf_train_list, ignore_index=True)
    for c in cat_cols:
        pdf_train[c] = pdf_train[c].astype('category')
    X_train, y_train = pdf_train[feat_cols], pdf_train['sales']
    
    model = LGBMRegressor(
        objective='regression',
        num_leaves=64,
        n_estimators=100,
        learning_rate=0.1
    )
    model.fit(X_train, y_train, categorical_feature=cat_cols)
    
    # Predict on all batches
    pred_results = []
    for pdf_pred in pdf_pred_list:
        for c in cat_cols:
            pdf_pred[c] = pdf_pred[c].astype('category')
        X_pred = pdf_pred[feat_cols]
        y_pred = np.maximum(0, np.round(model.predict(X_pred)))
        res = pdf_pred[['id', 'day_index']].copy()
        res['predicted_sales'] = y_pred
        pred_results.append(res)
    
    return pd.concat(pred_results, ignore_index=True)

In [0]:
pred_results = []
for cat in categories:
    print(f"--- Processing category: {cat} ---")
    df_cat = training_df.filter(col('cat_id') == cat)
    
    # Training data: day_index from train_start_idx to 1913
    train_df = df_cat.filter((col('day_index') >= train_start_idx) & (col('day_index') <= 1913))
    
    # Prediction data: day_index 1914–1969
    pred_df = df_cat.filter((col('day_index') >= 1914) & (col('day_index') <= 1969))
    
    # Process in batches
    batch_result = process_batch(train_df, pred_df, batch_size=5000)
    batch_result['category'] = cat
    pred_results.append(batch_result)

--- Processing category: FOODS ---


In [0]:
# === Combine Predictions ===
pred_all = pd.concat(pred_results, ignore_index=True)

In [0]:
# === Create Submission DataFrame ===
pred_spark = spark.createDataFrame(pred_all).select(
    col('id'),
    col('day_index'),
    col('predicted_sales').cast('int').alias('pred')
)

In [0]:
# === Pivot for Submission Format ===
sub_sdf = pred_spark.groupBy('id') \
    .pivot('day_index', list(range(1914, 1970))) \
    .agg(first('pred')) \
    .selectExpr(
        'id',
        # d1914-d1941 (F1-F28 for validation)
        *[f'`{d}` as F{i+1}' for i, d in enumerate(range(1914, 1942))],
        # d1942-d1969 (F1-F28 for evaluation)
        *[f'`{d}` as F{i+1}' for i, d in enumerate(range(1942, 1970))]
    )

In [0]:
# === Separate Validation and Evaluation Rows ===
val_sub_sdf = sub_sdf.filter(col('id').endswith('_validation')) \
    .select('id', *[f'F{i+1}' for i in range(28)])  # F1-F28 for d1914-d1941

eval_sub_sdf = sub_sdf.filter(col('id').endswith('_evaluation')) \
    .select('id', *[f'F{i+29}' for i in range(28)]).toDF('id', *[f'F{i+1}' for i in range(28)])  # F1-F28 for d1942-d1969

In [0]:
# === Union Validation and Evaluation ===
final_sub_sdf = val_sub_sdf.union(eval_sub_sdf)

java.net.NoRouteToHostException: No route to host
	at java.base/sun.nio.ch.Net.pollConnect(Native Method)
	at java.base/sun.nio.ch.Net.pollConnectNow(Net.java:672)
	at java.base/sun.nio.ch.SocketChannelImpl.finishConnect(SocketChannelImpl.java:946)
	at shaded.v9_4.org.eclipse.jetty.io.SelectorManager.doFinishConnect(SelectorManager.java:355)
	at shaded.v9_4.org.eclipse.jetty.io.ManagedSelector.processConnect(ManagedSelector.java:347)
	at shaded.v9_4.org.eclipse.jetty.io.ManagedSelector.access$1700(ManagedSelector.java:65)
	at shaded.v9_4.org.eclipse.jetty.io.ManagedSelector$SelectorProducer.processSelected(ManagedSelector.java:676)
	at shaded.v9_4.org.eclipse.jetty.io.ManagedSelector$SelectorProducer.produce(ManagedSelector.java:535)
	at shaded.v9_4.org.eclipse.jetty.util.thread.strategy.EatWhatYouKill.produceTask(EatWhatYouKill.java:362)
	at shaded.v9_4.org.eclipse.jetty.util.thread.strategy.EatWhatYouKill.doProduce(EatWhatYouKill.java:186)
	at shaded.v9_4.org.eclipse.jetty.util.thread.strategy.EatWhatYouKill.tryProduce(EatWhatYouKill.java:173)
	at shaded.v9_4.org.eclipse.jetty.util.thread.strategy.EatWhatYouKill.run(EatWhatYouKill.java:131)
	at shaded.v9_4.org.eclipse.jetty.util.thread.ReservedThreadExecutor$ReservedThread.run(ReservedThreadExecutor.java:409)
	at com.databricks.rpc.ShadedInstrumentedQueuedThreadPool$$anon$2.$anonfun$run$4(InstrumentedQueuedThreadPool.scala:183)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at com.databricks.logging.AttributionContextTracing.$anonfun$withAttributionContext$1(AttributionContextTracing.scala:49)
	at com.databricks.logging.AttributionContext$.$anonfun$withValue$1(AttributionContext.scala:293)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:62)
	at com.databricks.logging.AttributionContext$.withValue(AttributionContext.scala:289)
	at com.databricks.logging.AttributionContextTracing.withAttributionContext(AttributionContextTracing.scala:47)
	at com.databricks.logging.AttributionContextTracing.withAttributionContext$(AttributionContextTracing.scala:44)
	at com.databricks.rpc.ShadedInstrumentedQueuedThreadPool.withAttributionContext(InstrumentedQueuedThreadPool.scala:130)
	at com.databricks.rpc.ShadedInstrumentedQueuedThreadPool$$anon$2.$anonfun$run$3(InstrumentedQueuedThreadPool.scala:183)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at com.databricks.instrumentation.QueuedThreadPoolInstrumenter.trackActiveThreads(QueuedThreadPoolInstrumenter.scala:132)
	at com.databricks.instrumentation.QueuedThreadPoolInstrumenter.trackActiveThreads$(QueuedThreadPoolInstrumenter.scala:129)
	at com.databricks.rpc.ShadedInstrumentedQueuedThreadPool.trackActiveThreads(InstrumentedQueuedThreadPool.scala:130)
	at com.databricks.rpc.ShadedInstrumentedQueuedThreadPool$$anon$2.run(InstrumentedQueuedThreadPool.scala:177)
	at shaded.v9_4.org.eclipse.jetty.util.thread.QueuedThreadPool.runJob(QueuedThreadPool.java:883)
	at shaded.v9_4.org.eclipse.jetty.util.thread.QueuedThreadPool$Runner.run(QueuedThreadPool.java:1034)
	at java.base/java.lang.Thread.run(Thread.java:842)

In [0]:
# === Save Submission CSV ===
final_sub_sdf.coalesce(1).write.option('header', True).mode('overwrite') \
    .csv('/FileStore/submission_folder')
files = dbutils.fs.ls('/FileStore/submission_folder')
dbutils.fs.mv(
    [f.path for f in files if f.name.startswith('part-')][0],
    '/FileStore/submission.csv'
)
dbutils.fs.rm('/FileStore/submission_folder', recurse=True)

print('✅ /FileStore/submission.csv Finished')

In [0]:
# === Cleanup ===
training_df.unpersist()

--- Processing Validation for category: FOODS ---
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 11.879655 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2948
[LightGBM] [Info] Number of data points in the train set: 5625632, number of used features: 27
[LightGBM] [Info] Start training from score 1.803247
--- Processing Validation for category: HOUSEHOLD ---
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.829221 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2769
[LightGBM] [Info] Number of data points in the train set: 4117746, number of used features: 27
[LightGBM] [Info] Start training from score 0.920305
--- Processing Validation for category: HOBBIES ---
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.467